# Calculating rise/set times with astropy vs PyEphem

Calculate alt/az/RA/Dec/HA: 

$$ RA = HA - \arccos\left[{\sin(Alt) - \sin(Dec) \sin(Lat) \over \cos(Dec) \cos(Lat)}\right] $$
or
$$ HA = RA + \arccos\left[{\sin(Alt) - \sin(Dec) \sin(Lat) \over \cos(Dec) \cos(Lat)}\right] $$
References: see [here](http://en.wikibooks.org/wiki/General_Astronomy/Coordinate_Systems) and [here](http://en.wikipedia.org/wiki/Celestial_coordinate_system#Equatorial_.E2.86.90.E2.86.92_horizontal)

In [205]:
# With astropy: 

import numpy as np
from astropy.time import Time
import astropy.units as u
import datetime

alt = 0.0
RA = 15*u.deg
Dec = 15*u.deg
latitude = 0*u.deg
longitude = 0*u.deg
date = Time('2014-05-12 00:00:00', format='iso')

def getHA(RA, Dec, lat, alt):
    return np.arccos( (np.sin(alt) - np.sin(Dec)*np.sin(lat))
                         / (np.cos(Dec)*np.cos(lat)) ).to(u.hourangle)

def previous_rise_set_transit(HA, RA, time, longitude, kind='apparent', offset=0):
    '''
    time : instance of `astropy.time.Time`
    '''
    lst = time.sidereal_time(kind, longitude)
    
    # Previous transit was this many hours ago
    previous_transit_deltaHA =  lst - RA.to(u.hourangle)
    previous_transit = (time.datetime - datetime.timedelta(
                        hours=float(previous_transit_deltaHA/u.hourangle)) +
                        datetime.timedelta(days=offset))
    rise_time = previous_transit - datetime.timedelta(hours=float(HA/u.hourangle))
    set_time = previous_transit + datetime.timedelta(hours=float(HA/u.hourangle))
    return rise_time, set_time, previous_transit

HA = getHA(RA, Dec, lat, alt)

print("Rise: {0}\nSet: {1}\nTransit: {2}".format(
        *previous_rise_set_transit(HA, RA,date, longitude)))

Rise: 2014-05-11 03:41:14.675896
Set: 2014-05-11 15:41:14.675896
Transit: 2014-05-11 09:41:14.675896


In [206]:
# With PyEphem:
import ephem

veq = ephem.FixedBody()
veq._ra = ephem.hours('01:00:00') # hours!
veq._dec = ephem.degrees('15:00:00')
o = ephem.Observer()
o.lat = 0
o.lon = 0
o.date = date.iso
veq.compute(o)
risetime = o.previous_rising(veq)
settime = o.previous_setting(veq)
trans = o.previous_transit(veq)

print 'Rise:', risetime
print 'Set:', settime
print 'Transit:', trans


Rise: 2014/5/11 03:43:00
Set: 2014/5/11 15:45:40
Transit: 2014/5/11 09:44:20


**NOTE** This astropy solution only works for dates before the present, since the astropy.time.Time.sidereal_time() calls the [IERS B table](https://github.com/astropy/astropy/blob/master/astropy/utils/iers/iers.py#L34) by [design](http://astropy.readthedocs.org/en/latest/_modules/astropy/time/core.html#Time.sidereal_time) and can't be switched to the B table with a keyword argument (yet).